# **🔁 Recurrent Neural Networks**

---

## 🔗 Sequential Input
- Designed for **sequential data**: text, time series, speech, etc.
- Processes input **step-by-step**, maintaining memory of past inputs.

---

## 🧠 Recurrent Neurons
- Each neuron takes input **and** previous output (hidden state).
- Has a feedback loop that enables **memory**.
- Formula: `hₜ = f(Wxₜ + Uhₜ₋₁ + b)`

---

## ⛓️ Hidden State
- Captures **context** from previous time steps.
- Passed from one time step to the next.
- Enables learning of time-dependent patterns.

---

## 🔢 Input / Output
- Input: A sequence (e.g., sentence = [word1, word2, ...])
- Output can be:
  - One-to-One (e.g. sentiment classification)
  - One-to-Many (e.g. image captioning)
  - Many-to-One (e.g. sequence classification)
  - Many-to-Many (e.g. translation)

---

## 🧮 Activation Function
- Commonly uses **tanh** or **ReLU** for hidden state.
- **Softmax** at output for classification tasks.

---

## ♻️ Backpropagation Through Time (BPTT)
- Extension of backpropagation for RNNs.
- Unrolls the network through time steps.
- Adjusts weights by computing gradients over all steps.

---

## ⚠️ Challenges
- **Vanishing/exploding gradients** in long sequences.
- Solved using advanced RNNs like:
  - **LSTM (Long Short-Term Memory)** 📦
  - **GRU (Gated Recurrent Unit)** 🔐

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [ ]:
# Dataset

data = pd.read_csv("monthly_milk_production.csv")
data.columns = ['Date','Production']
data.head()

In [ ]:
# Scaling

data["Date"] = pd.to_datetime(data["Date"])
data.set_index("Date", inplace=True)
production = data["Production"].astype(float).values.reshape(-1, 1)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(production)

In [ ]:
# Sequencing

window_size = 12
X = []
y = []
target_dates = data.index[window_size:]

for i in range(window_size, len(scaled_data)):
    X.append(scaled_data[i - window_size : i, 0])
    y.append(scaled_data[i, 0])

X = np.array(X)
y = np.array(y)

In [ ]:
# Model

X_train, X_test, y_train, y_test, dates_train, dates_test = train_test_split(X, y, target_dates, test_size=0.2, random_state=42, shuffle=False)

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=128))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer="adam", loss="mean_squared_error")
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1)

In [ ]:
# Predictions

predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions).flatten()
y_test = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

rmse = np.sqrt(np.mean((y_test - predictions) ** 2))
print(f"RMSE: {rmse:.2f}")

In [ ]:
# Plot

plt.figure(figsize=(12, 6))
plt.plot(dates_test, y_test, label="Actual Production")
plt.plot(dates_test, predictions, label="Predicted Production")
plt.title("Actual vs Predicted Production")
plt.xlabel("Date")
plt.ylabel("Production (pounds per cow)")
plt.legend()
plt.show()